In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../')
sys.argv=[''] 
del sys

In [3]:
cd "/root/SymmNeRF-baseline/code"

/root/SymmNeRF-baseline/code


In [4]:
import setproctitle
from opt import config_parser
import torch 
import numpy as np 
import matplotlib.pyplot as plt 
import setproctitle
import tqdm 
from utils.general import * 
parser = config_parser()
args = parser.parse_args()

device = torch.device("cuda:1")
args.archCNN = 'resnet34'

det = args.det
lindisp = args.lindisp
args.distributed = False
args.local_feature_ch = 512
args.local_rank = 1
args.eval_scene = 'cars'
args.eval_dataset = 'srns_dataset'
args.num_local_layers = 2
args.no_load_opt = True
args.no_load_scheduler = True


args.enforce_symmetry = {'status':False,
                         'on_coarse':False,'on_fine':False,
                         'concatenate_on_coarse':False, 'concatenate_on_fine':False}

args.module_with_cosine = {'use':False,'G':16}

setproctitle.setproctitle('[Gaetan - DeepLabV3 encoder]')

In [5]:
from model.model import HyperNeRFResNetSymmLocal 

args.use_deepLabv3 = True
args.discard_hypernetworks = True

model= HyperNeRFResNetSymmLocal(args,ckpts_folder = '')

[INFO] Using ResNet MLP to generate the weights of the NeRF. [INFO]
[INFO] Using DeepLabV3 as CNN feature extractor. [INFO]
[Info] No ckpts found, training from scratch...


In [6]:
from datasets import dataset_dict, create_training_dataset
from torch.utils.data import DataLoader 

bs = 2

train_dataset, train_sampler = create_training_dataset(args)
train_loader = DataLoader(train_dataset,batch_size = bs , sampler = train_sampler,shuffle=True)

[Info] Training dataset: srns_dataset
[Info] Loading SRNs dataset: /data/srn_cars/cars_train


In [7]:
from model.sample_ray import RaySampler
from utils.general import cycle

# Get another iteration on the Dataloader
it = iter(train_loader)
train_batch = it.next()

# RaySampler class instantiation.
ray_sampler = RaySampler(train_batch) 

# Number of rays and number of points sampled alogn each ray.
nb_rays = 64
nb_sampled_points_on_rays = 256

# Build up a batch of ray. 
ray_batch = ray_sampler.random_sample(nb_rays,use_bbox = True,device = device)

/usr/local/lib/python3.9/dist-packages/torch/functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [8]:
from model.render_ray import sample_along_camera_ray

rays_o = ray_batch['rays_o']    # [B,256,3]
rays_d = ray_batch['rays_d']    # [B,256,3]
z_near = ray_batch['z_near']
z_far = ray_batch['z_far']

noise = False 

#### 1. Get the 3D points ray, the viewdir and z sampled dist.
pts, viewdirs, z_vals = sample_along_camera_ray(rays_o=rays_o,  # pts : [B,256,64,3] - viewdirs : [B,256,3] - z_vals : [B,256,64]
                                                rays_d=rays_d,
                                                z_near=z_near,
                                                z_far=z_far,
                                                device=device,
                                                N_samples=nb_sampled_points_on_rays,
                                                lindisp=lindisp, 
                                                det=det)
z = model.encode(ray_batch['src_img'])
ret = model.feature_net.index(pts,ray_batch)

print(pts.device)

cuda:1


In [15]:
from model.render_ray import get_symmetric_points_and_directions

print(f'Shape of viewdirs: {viewdirs.shape}')
print(f'Shape of pts: {pts.shape}')

M = model.get_symmetry_matrix()
      
ptsS,viewdirsS = get_symmetric_points_and_directions(pts,viewdirs,M)
print(rays_d[...,None,:].shape)

Shape of viewdirs: torch.Size([2, 64, 3])
Shape of pts: torch.Size([2, 64, 256, 3])
torch.Size([2, 64, 1, 3])


In [9]:
from einops import rearrange,repeat

N_rays = rays_d.shape[1]
batch = rays_d.shape[0]
        
rgb = ret['rgb']
f_coarse = ret['feat']
f_coarse_s = ret['feat_s']

xyz_c = ret['xyz_c'] # 3D points sampled from target view expressed in source view coordinates.
xyz_c = rearrange(xyz_c,'b (nr ns) ch -> b nr ns ch', nr=nb_rays,ns= nb_sampled_points_on_rays)
        
dirs = repeat(ray_batch['rays_d'], 'b nr c -> b nr ns c', ns=nb_sampled_points_on_rays)
dir_c = model.feature_net.compute_dir(dirs, ray_batch)  
        
rgb= rearrange(rgb,'b (nr ns) rgb -> b nr ns rgb', nr=nb_rays,ns= nb_sampled_points_on_rays)
f_coarse = rearrange(f_coarse,'b (nr ns) c -> b nr ns c', nr=nb_rays,ns= nb_sampled_points_on_rays)
f_coarse_s = rearrange(f_coarse_s,'b (nr ns) c -> b nr ns c', nr=nb_rays,ns= nb_sampled_points_on_rays)
     
# Compute the symmetric points and directions
feat = torch.cat([f_coarse,f_coarse_s,rgb,dir_c],-1)

print(feat.shape)

torch.Size([2, 64, 256, 518])


In [10]:
raw_coarse = model.coarse_net(xyz_c.flatten(0, 1), feat.flatten(0, 1))   # [batch*N_rays*N_samples, 4]
raw_coarse = raw_coarse.reshape([batch, nb_rays, nb_sampled_points_on_rays, 4])        

In [9]:
from model.render_ray import render_rays
cosine_mod = {'use':False}
ret_out = render_rays(ray_batch,model,device,z,{},nb_sampled_points_on_rays,cosine_mod,use_ray_transformer=False,N_importance=64)

Device for pts: cuda:1


../aten/src/ATen/native/cuda/ScatterGatherKernel.cu:111: operator(): block: [26,0,0], thread: [96,0,0] Assertion `idx_dim >= 0 && idx_dim < index_size && "index out of bounds"` failed.
../aten/src/ATen/native/cuda/ScatterGatherKernel.cu:111: operator(): block: [26,0,0], thread: [97,0,0] Assertion `idx_dim >= 0 && idx_dim < index_size && "index out of bounds"` failed.
../aten/src/ATen/native/cuda/ScatterGatherKernel.cu:111: operator(): block: [26,0,0], thread: [102,0,0] Assertion `idx_dim >= 0 && idx_dim < index_size && "index out of bounds"` failed.
../aten/src/ATen/native/cuda/ScatterGatherKernel.cu:111: operator(): block: [26,0,0], thread: [103,0,0] Assertion `idx_dim >= 0 && idx_dim < index_size && "index out of bounds"` failed.
../aten/src/ATen/native/cuda/ScatterGatherKernel.cu:111: operator(): block: [26,0,0], thread: [114,0,0] Assertion `idx_dim >= 0 && idx_dim < index_size && "index out of bounds"` failed.
../aten/src/ATen/native/cuda/ScatterGatherKernel.cu:111: operator(): blo

NameError: name 'exit' is not defined

In [36]:
print(ret_out['outputs_fine'].keys())

AttributeError: 'NoneType' object has no attribute 'keys'